In [ ]:
from ocpy import ddp, symutils
from ocpy.ddp import DDPSolver
from ocpy.ocp import OCP
import numpy as np
import sympy as sym
from sympy import sin, cos, tan, exp, log, ln, sinh, cosh, tanh, diff, sqrt

# import warnings
# warnings.filterwarnings('error')


In [ ]:
# dimensions of state and input
n_x = 4
n_u = 1
sim_name = 'cartpole'
# define ocp class
ocp = OCP(n_x, n_u, sim_name)
# get symbols
t = ocp.get_t()
x = ocp.get_x()
u = ocp.get_u()

In [ ]:
# symbolic expressions of constants.
m_c, m_p, l, g, u_min, u_max, u_eps = ocp.define_scalar_constants(
         [('m_c', 2), ('m_p', 0.1), ('l', 0.5), ('g', 9.80665), 
          ('u_min', -15),  ('u_max', 15), ('u_eps', 0.001)]
          )
# cost weight
q = ocp.define_vector_constant('q', [2.5, 10, 0.01, 0.01])
r = ocp.define_vector_constant('r', [1])
q_f = ocp.define_vector_constant('q_f', [2.5, 10, 0.01, 0.01])
Q = sym.diag(*q)
Qf = sym.diag(*q_f)
R = sym.diag(*r)
# reference state. 
x_ref = ocp.define_vector_constant('x_ref', [0, np.pi, 0, 0])


In [ ]:
# state of equation.
f = ocp.get_f_empty()
f[0] = x[2]
f[1] = x[3]
f[2] = (u[0] + m_p*sin(x[1])*(l*x[1]*x[1] + g*cos(x[1])) )/( m_c+m_p*sin(x[1])*sin(x[1]) )
f[3] = (-u[0] * cos(x[1]) - m_p*l*x[1]*x[1]*cos(x[1])*sin(x[1]) 
        - (m_c+m_p)*g*sin(x[1]) )/( l*(m_c + m_p*sin(x[1])*sin(x[1])))
# log barrier for control limits.
u_barrier = sum(-ln(u[i] - u_min) - ln(u_max - u[i]) for i in range(n_u)) * 1e-5
# stage cost and terminal cost.
l = (x - x_ref).T * Q * (x - x_ref) + u.T * R * u + sym.Matrix([u_barrier])
lf = (x - x_ref).T * Qf * (x - x_ref) * 10
# display state equation and cost function
display(f)
display(l)
display(lf)


In [ ]:
# horizon length and discretization grids.
T = 5.0
N = 200
# initial condition
t0 = 0.0
x0 = np.array([0.0, 0.0, 0.0, 0.0])
us_guess = np.zeros((N, n_u))
# define ocp
ocp.define(f, l, lf, T, N, t0, x0, us_guess, True, False)


In [ ]:
# hand over ocp to solver.
solver = DDPSolver(ocp)

In [ ]:
# warnings.filterwarnings('error')
# warnings.resetwarnings()
# candidates of line search steps.
alphas = [0.5**i for i in range(8)].append(0)
# set solver properties.
solver.set_solver_parameters(
    max_iter=250, alphas=alphas, damp_init=1e-3, damp_max=1e4, damp_min=1e-3
)
# solve ocpj
ts, xs, us, Js = solver.solve(result= True, log=True)

# damp_fixed = 0
# ts, xs, us, Js = solver.solve(
#     t0, x0, us_guess, max_iter=300, alphas=0.5**np.arange(1) ,
#     damp_init=damp_fixed, damp_min=damp_fixed, damp_max=damp_fixed, result=True, log=True)

In [ ]:
%matplotlib inline
# visualize
from ocpy.animator import CartPoleAnimator
animator = CartPoleAnimator(solver.get_log_directory(), sim_name)
animator.generate_animation(False)


In [ ]:
ocp.get_symbolic_derivatives_substituted()[1][0]

In [ ]:
ocp.get_symbolic_derivatives()[0][1]